In [1]:
from subprocess import Popen, PIPE, STDOUT
from random import randint
import binascii
import os
from queue import Queue, Empty
from threading  import Thread
from time import sleep
import random
import nltk
nltk.download('averaged_perceptron_tagger')
from itertools import permutations, combinations
import spacy
from spacy.matcher import Matcher
from spacy.attrs import POS
nlp = spacy.load('en')
nlp = spacy.load('en_core_web_sm')
import pandas as pd

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sparr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
basic_actions = [
           'go north',
           'go south',
           'go west',
           'go east',
           'go northeast',
           'go northwest',
           'go southeast',
           'go southwest',
           'd',
           'u'
]

action_space = set()

for action in basic_actions:
    action_space.add(action)

command1_actions = ['get OBJ',
 'set OBJ',
 'hit OBJ',
 'eat OBJ',
 'put OBJ',
 'cut OBJ',
 'dig OBJ',
 'shout',
 'ask OBJ',
 'fix OBJ',
 'give OBJ',
 'make OBJ',
 'wear OBJ',
 'move OBJ',
 'kick OBJ',
 'kill OBJ',
 'find OBJ',
 'play OBJ',
 'feel OBJ',
 'hide OBJ',
 'read OBJ',
 'fill OBJ',
 'go OBJ',
 'flip OBJ',
 'burn OBJ',
 'pick OBJ',
 'pour OBJ',
 'pull OBJ',
 'apply OBJ',
 'leave OBJ',
 'ask OBJ',
 'break OBJ',
 'enter OBJ',
 'curse OBJ',
 'shake OBJ',
 'carry OBJ',
 'spray OBJ',
 'burn OBJ',
 'inflat OBJ',
 'brandi OBJ',
 'donate OBJ',
 'squeez OBJ',
 'attach OBJ',
 'find OBJ',
 'unlock OBJ',
 'banish OBJ',
 'read OBJ',
 'wait OBJ',
 'enchan OBJ',
 'feel OBJ',
 'pour OBJ']
                  

command2_actions = ['pour OBJ on DCT',
 'hide OBJ in DCT',
 'pour OBJ in DCT',
 'move OBJ in DCT',
 'hide OBJ on DCT',
 'flip OBJ for DCT',
 'fix OBJ with DCT',
 'spray OBJ on DCT',
 'dig OBJ with DCT',
 'cut OBJ with DCT',
 'pick OBJ with DCT',
 'squeez OBJ on DCT',
 'pour OBJ from DCT',
 'fill OBJ with DCT',
 'brandi OBJ at DCT',
 'burn OBJ with DCT',
 'flip OBJ with DCT',
 'read OBJ with DCT',
 'hide OBJ under DCT',
 'carry OBJ from DCT',
 'inflat OBJ with DCT']
                    

filtered_tokens = ['Score', 'Moves']

In [3]:
def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

In [4]:
def start_game():
    score = 0
    moves = 0
    p = Popen(['dfrotz.exe','zork1.z5'], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    q = Queue()
    t = Thread(target=enqueue_output, args=(p.stdout, q))
    t.daemon = True # thread dies with the program
    t.start()
    sleep(0.1)
    return(p, q, t, score, moves)

In [27]:
# read line without blocking
def readLine(q):
    cont = True
    narrative = ""
    while cont:
        try:  line = q.get_nowait() # or q.get(timeout=.1)
        except Empty:
            cont = False
        else: 
            narrative = narrative + line.decode("utf-8").replace('\n', " ").replace('\r', " ")
    if ('840726' in narrative): ## Opening narrative
        narrative = narrative[narrative.index('840726') + len('840726'):]
    try:
        score, moves = grab_score_moves(narrative)
        narrative = narrative[narrative.index('Moves: ')+len('Moves:')+3:-1].strip()
    except:  ## not valid move
        pass
    sleep(0.1)
    return(narrative, score, moves)

In [6]:
def look_surroundings(p):
    perform_action('look', p)

In [7]:
def check_inventory(p):
    perform_action('inventory', p)

In [8]:
def grab_score_moves(narrative):
    try:
        score = int(narrative[narrative.index('Score: ') + len('Score: '):][0:3].strip())
        moves = int(narrative[narrative.index('Moves: ') + len('Moves: '):][0:3].strip())
    except:  ## not valid move
        score = 0
        moves = 0
    return(score, moves)

In [9]:
def preprocess(text): 
    text = nltk.word_tokenize(text)
    processed = nltk.pos_tag(text)
    return processed

In [10]:
def detect_nouns(narrative):  #grabs nouns and adjective noun pairs from narrative
    
    processed = preprocess(narrative)
    pattern = 'NP: {<JJ>*<NN>}'
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(processed)
    noun_list =  [noun.text for noun in nlp(narrative).noun_chunks]
    return(noun_list)
    
    # function to test if something is a noun
    #is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    #tokenized = nltk.word_tokenize(narrative)
    #nouns = [word.lower() for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    #return(nouns)

In [11]:
def get_nouns(narrative):
    matcher = Matcher(nlp.vocab)
    matcher.add('Noun phrase', None, [{POS: 'NOUN'}])
    doc = nlp(narrative)
    matches = matcher(doc)
    noun_list = [doc[start:end].text for id, start, end in matches]
    return(noun_list)

In [12]:
def generate_action_tuples(nouns):
    possible_actions = []
    for i in nouns:
        for action1 in command1_actions:   ## first loop replaces 'x' in each action in command1_actions
            action_to_add = action1.replace('OBJ', i)
            possible_actions.append(action_to_add)
        noun_permutations = list(permutations(nouns, 2))    ## second loop replaces 'x' and 'y' in each action in command2_actions
        for action2 in command2_actions:
            for perm in noun_permutations:
                if (perm[0] == perm[1]):  ## ignore same noun acting on itself
                    pass
                else:
                    action_to_add = action2.replace('OBJ', perm[0])
                    action_to_add = action_to_add.replace('DCT', perm[1])
                    possible_actions.append(action_to_add)
    return possible_actions

In [13]:
def generate_action_space(nouns):
    possible_actions = []
    for i in nouns:
        for action1 in command1_actions:   ## first loop replaces 'x' in each action in command1_actions
            action_to_add = action1.replace('x', i)
            possible_actions.append(action_to_add)
        noun_permutations = list(permutations(nouns, 2))    ## second loop replaces 'x' and 'y' in each action in command2_actions
        for action2 in command2_actions:
            for perm in noun_permutations:
                if (perm[0] == perm[1]):  ## ignore same noun acting on itself
                    pass
                else:
                    action_to_add = action2.replace('x', perm[0])
                    action_to_add = action_to_add.replace('y', perm[1])
                    possible_actions.append(action_to_add)
    return possible_actions

In [14]:
def add_to_action_space(action_space, actions):
    for action in actions:
        action_space.add(action)
    return action_space

In [15]:
def perform_action(command, p):
    p.stdin.write(bytes(command+ "\n", 'ascii'))
    p.stdin.flush()
    sleep(0.1)## wait for action to register

In [16]:
def kill_game(p):
    p.terminate()
    p.kill()

In [17]:
def choose_random_action(p, q, action_space):
    action = random.sample(action_space, 1)
    return action[0]

In [77]:
def generate_embeddings(text):
    text = word_tokenize(text)
    model = Word2Vec(text, size=16)
    words = list(model.wv.vocab)
    print(words)

In [80]:
p, q, t, score, moves = start_game()
moves_count = 0
story = pd.DataFrame(columns=['Surroundings', 'Inventory', 'Action', 'Response', 'Score', 'Moves'])
for i in range(0,10):
    ## Check surroundings, check inventory, choose action, check action response
    
    narrative,score,moves = readLine(q)
    check_inventory(p)
    inventory,s,m = readLine(q)
    
    nouns = get_nouns(narrative)
    current_action_space = generate_action_tuples(nouns)
    action_space = add_to_action_space(action_space, current_action_space)
    action = choose_random_action(p,q, basic_actions)
    if i==0:
        perform_action('open mailbox',p)
    elif i==1:
        perform_action('take leaflet',p)
    else:
        perform_action(action, p)
    
    response,score,moves = readLine(q)
        
    story.loc[i] = [narrative, inventory, action, response, s, i+1]
    
    generate_embeddings(story['Response'][i])
    
    look_surroundings(p)

kill_game(p)

['e', 'a', 'l']


RuntimeError: you must first build vocabulary before training the model

In [60]:
story

,Surroundings,Inventory,Action,Response,Score,Moves
0,,You are empty-handed.,go northwest,Opening the small mailbox reveals a leaflet.,0,1
1,West of House You are standing in an open fie...,You are empty-handed.,go north,Taken.,0,2
2,West of House You are standing in an open fie...,You are carrying: A leaflet,go west,"Forest This is a forest, with trees in all di...",0,3
3,"Forest This is a forest, with trees in all di...",You are carrying: A leaflet You hear in th...,go southeast,You can't go that way.,0,4
4,"Forest This is a forest, with trees in all di...",You are carrying: A leaflet,go southeast,You can't go that way.,0,5
5,"Forest This is a forest, with trees in all di...",You are carrying: A leaflet You hear in th...,go south,"Forest This is a dimly lit forest, with large...",0,6
6,"Forest This is a dimly lit forest, with large...",You are carrying: A leaflet,go east,The rank undergrowth prevents eastward movement.,0,7
7,"Forest This is a dimly lit forest, with large...",You are carrying: A leaflet,go southwest,You can't go that way.,0,8
8,"Forest This is a dimly lit forest, with large...",You are carrying: A leaflet,go northeast,You can't go that way.,0,9
9,"Forest This is a dimly lit forest, with large...",You are carrying: A leaflet,go southeast,You can't go that way. You hear in the distan...,0,10


In [81]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [82]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

KeyboardInterrupt: 

In [76]:
word_tokenize(story['Surroundings'][9])

['South',
 'of',
 'House',
 'You',
 'are',
 'facing',
 'the',
 'south',
 'side',
 'of',
 'a',
 'white',
 'house',
 '.',
 'There',
 'is',
 'no',
 'door',
 'here',
 ',',
 'and',
 'all',
 'the',
 'windows',
 'are',
 'boarded',
 '.']

In [109]:
kill_game(p)


In [39]:
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import sent_tokenize, word_tokenize

In [40]:
model = gensim.models.KeyedVectors.load_word2vec_format('lexvec.enwiki+newscrawl.300d.W+C.pos.vectors', binary=True) 

In [57]:
model.wv.most_similar(positive=['water'])

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('.591279', 0.40431690216064453),
 ('26031', 0.3972581624984741),
 ('.902732', 0.3782956302165985),
 ('.231285', 0.3713010251522064),
 ('8701', 0.36930298805236816),
 ('-0.680219', 0.3668520450592041),
 ('179443', 0.36293667554855347),
 ('12129', 0.35880768299102783),
 ('812944', 0.3574475646018982),
 ('-1.046399', 0.35737481713294983)]